#**YQuantum 2025 Challenge : Team Polytrit Panacea**

Quantum Hash Function


In [ ]:
!pip install qiskit

## Step 1: Define the Hash Function Circuit

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

def input_to_angles(bitstring):
    return [np.pi if b == '1' else 0 for b in bitstring]

def quantum_hash_function(input_bits):
    angles = input_to_angles(input_bits)
    qc = QuantumCircuit(4, 4)

    for i in range(4):
        qc.ry(angles[i], i)

    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)

    qc.h(0)
    qc.rz(np.pi/4, 1)
    qc.rx(np.pi/3, 2)
    qc.ry(np.pi/5, 3)
    qc.cz(0, 3)

    qc.measure(range(4), range(4))

    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1024)
    result = job.result()
    counts = result.get_counts()

    return counts

# Generate all 4-bit binary strings
all_inputs = [''.join(bits) for bits in product('01', repeat=4)]

# Run the hash function on all inputs and store results
all_counts = {inp: quantum_hash_function(inp) for inp in all_inputs}

# Plotting
fig, axs = plt.subplots(4, 4, figsize=(18, 14))
fig.suptitle('Quantum Hash Outputs for All 4-bit Inputs', fontsize=20)

for idx, inp in enumerate(all_inputs):
    row, col = divmod(idx, 4)
    ax = axs[row, col]
    plot_histogram(all_counts[inp], ax=ax, title=f"Input: {inp}", bar_labels=False)

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()


## Step 2: Preimage & Collision Resistance Analysis

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine
from itertools import combinations

# Convert counts to probability vectors over all possible 4-bit outputs
def counts_to_prob_vector(counts, shots=1024):
    all_outputs = [format(i, '04b') for i in range(16)]
    return np.array([counts.get(k, 0) / shots for k in all_outputs])

# Compute cosine similarities between every pair of hash outputs
similarity_matrix = []
collisions = []

inputs = list(all_counts.keys())
prob_vectors = {inp: counts_to_prob_vector(all_counts[inp]) for inp in inputs}

for a, b in combinations(inputs, 2):
    sim = 1 - cosine(prob_vectors[a], prob_vectors[b])
    similarity_matrix.append((a, b, sim))
    if sim > 0.95:  # high similarity = potential collision
        collisions.append((a, b, sim))

# Show potential collisions
print("🔁 Potential Collisions (Cosine Similarity > 0.95):")
for a, b, sim in collisions:
    print(f"Inputs {a} and {b} -> Similarity: {sim:.4f}")

# Show average similarity (lower is better)
avg_sim = np.mean([sim for _, _, sim in similarity_matrix])
print(f"\n📉 Average Pairwise Cosine Similarity: {avg_sim:.4f}")


## Step 3: Entropy & Randomness of Hash Output

In [ ]:
from scipy.stats import entropy
import matplotlib.pyplot as plt

entropy_values = {}

for inp, probs in prob_vectors.items():
    ent = entropy(probs, base=2)  # log base 2 = bits
    entropy_values[inp] = ent

# Print entropy values
print("🧮 Entropy of each input's hash output:")
for inp, ent in entropy_values.items():
    print(f"Input {inp}: Entropy = {ent:.4f} bits")

# Plotting entropy
plt.figure(figsize=(10, 5))
plt.bar(entropy_values.keys(), entropy_values.values(), color='skyblue')
plt.ylim(0, 4)  # max for 4-bit output
plt.xlabel("Input Bitstring")
plt.ylabel("Shannon Entropy (bits)")
plt.title("Entropy of Quantum Hash Outputs")
plt.grid(True)
plt.show()

# Summary
avg_entropy = np.mean(list(entropy_values.values()))
print(f"\n📊 Average Entropy Across All Inputs: {avg_entropy:.4f} bits")


## Step 4: Scalability and Circuit Optimization

In [ ]:
# Scaling to N qubits and running entropy analysis for larger inputs
def simulate_hash_with_qubits(n_qubits):
    """ Simulate the hash function for varying numbers of qubits. """
    inputs = [bytes(np.random.randint(0, 256, n_qubits)) for _ in range(100)]
    prob_vectors = {}

    for inp in inputs:
        hash_output = simple_quantum_hash(inp)
        prob_vectors[inp] = calc_probability_distribution(hash_output)

    # Calculate entropy for each hash output
    entropy_values = {inp: entropy(probs, base=2) for inp, probs in prob_vectors.items()}
    return entropy_values

# Test with 10 qubits, 15 qubits, and 20 qubits
qubit_tests = [10, 15, 20]
all_entropy = {}

for qubits in qubit_tests:
    entropy_values = simulate_hash_with_qubits(qubits)
    avg_entropy = np.mean(list(entropy_values.values()))
    all_entropy[qubits] = avg_entropy

# Plot the entropy comparison for different qubit numbers
plt.figure(figsize=(10, 6))
plt.plot(all_entropy.keys(), all_entropy.values(), marker='o', color='green')
plt.xlabel("Number of Qubits")
plt.ylabel("Average Entropy (bits)")
plt.title("Entropy vs Number of Qubits")
plt.grid(True)
plt.show()

# Print Entropy comparison
for qubits, entropy_val in all_entropy.items():
    print(f"With {qubits} qubits, the average entropy is {entropy_val:.4f} bits")


## Step 5: Final Testing and Profiling

In [ ]:
import time

# Test the hash function for 10 qubits and measure time
start_time = time.time()
simulate_hash_with_qubits(10)
end_time = time.time()
print(f"Execution time for 10 qubits: {end_time - start_time:.5f} seconds")
